In [58]:
import numpy as np
from itertools import cycle
from math import pi
from scipy.optimize import minimize
from scipy.linalg import expm, sinhm, coshm
import matplotlib.pyplot as plt
import time
from tqdm import tqdm

In [84]:
class Variational_amplitude_embeding:
         
    def __init__(self, n_qubits):
        
        self.n = n_qubits
        self.X = self.new_X()
        self.Z = self.new_Z()
               
    def new_Z(self):
        bin_a = lambda q,n_q: list([0]*2**(n_q-q) + [1]*2**(n_q-q))*2**(q-1) #creates strings of 0s and 1s
        z_list = []
        m_ones = -np.ones(2**self.n)
        for j in range(1,self.n+1):
            z_list.append(m_ones**np.array(bin_a(j,self.n)))
        return z_list
        
    def new_X(self):
        def split(x,k):
            return x.reshape((2**k,-1))
        def sym_swap(x):
            return np.asarray([x[-1],x[-2],x[1],x[0]])
        
        n = self.n
        x_list = []
        t1 = np.asarray([np.arange(2**(n-1),2**n),np.arange(0,2**(n-1))])
        t1 = t1.flatten()
        x_list.append(t1.flatten())
        t2 = t1.reshape(4,-1)
        t3 = sym_swap(t2)
        t1 = t3.flatten()
        x_list.append(t1)   
        
        k = 1
        while k < (n-1):
            t2 = split(t1,k)
            t2 = np.asarray(t2)
            t1=[]
            for y in t2:
                t3 = y.reshape((4,-1))
                t4 = sym_swap(t3)
                t1.append(t4.flatten())
            t1 = np.asarray(t1)
            t1 = t1.flatten()
            x_list.append(t1)
            k+=1        
        
        return x_list

    def x (self, state, ind):
        ind_x = self.X[ind]
        return state[ind_x]
        
    def y (self, state, ind):
        ind_x = self.X[ind]
        sign_z = self.Z[ind]
        return 1j*(sign_z * state)[ind_x]
    
    def z (self, state, ind):
        sign_z = self.Z[ind]
        return sign_z*state
    
    def rx (self, state, theta, ind):
        state_c = np.copy(state)
        state_X = self.x(state_c, ind)
        state_Rx = np.cos(theta)*state-1j*np.sin(theta)*state_X
        return state_Rx

    def ry (self, state, theta, ind):
        state_c = np.copy(state)
        state_Y = self.y(state_c, ind)
        state_Ry = np.cos(theta)*state-1j*np.sin(theta)*state_Y
        return state_Ry

    def rz (self, state, theta, ind):
        state_c = np.copy(state)
        state_Z = self.z(state_c, ind) 
        state_Rz = np.cos(theta)*state-1j*np.sin(theta)*state_Z
        return state_Rz

    def rxx (self, state, theta, ind1, ind2):
        state_c = np.copy(state)
        state_X = self.x(state_c, ind1)
        state_X = self.x(state_X, ind2)
        state_Rx = np.cos(theta)*state-1j*np.sin(theta)*state_X
        return state_Rx

######################## Ansatz circuit ################################### 

    def ising_entangler (self, state, pc, ind1, ind2): # realistic block
        state = self.rz(state, next(pc), ind1)
        state = self.rz(state, next(pc), ind2)
        state = self.rxx(state, next(pc), ind1, ind2)
        state = self.rx(state, next(pc), ind1)
        state = self.rx(state, next(pc), ind2)
        return state
    
    def checkerboard (self, state, p, pc):
        for _ in range(p):
            for ind_1 in range(self.n//2):
                state = self.ising_entangler(state, pc, 2*ind_1, 2*ind_1+1)
            for ind_2 in range(self.n//2-1):
                state = self.ising_entangler(state,pc,2*ind_2+1,2*ind_2+2)
            state = self.ising_entangler(state, pc, 0, self.n-1)
        return state
    
############################# Optimization ###################################

    def expected (self, state, H, depth, params): # expectation value of an ansatz
        pc = cycle(params)# parmeters iterator
        psi = self.checkerboard(state, depth, pc)
        return np.real(psi.conj().dot(H).dot(psi))
       
    def ex_minimize (self, init_state, H, depth, optimizer = 'L-BFGS-B', op_reps = 10, maxiter = 100, bds_1_block=([(0,2*np.pi)]*5)):
        self.init_state = init_state     
        self.H = H
        self.p = depth
        
        def cost (params): # cost function, to minimize expectation value 
            ex = self.expected(self.init_state, self.H, self.p, params) 
            return ex           

        num_blocks = self.rand_params(self.p)[2]
        bds = bds_1_block*num_blocks # boundaries for the whole ansatz

        best = 1
        
        t0 = time.time()
        for r in range(op_reps):
            params = self.rand_params(self.p)[0] 
            res =  minimize(cost, params, method = optimizer, jac=None, options = {'maxiter': maxiter}, bounds = bds) #THIS IS THE OPTIMIZER

            if res.fun < best:
                best = res.fun
                opt_params = res.x
            
        self.time_tot = time.time()-t0
        
        self.ex_val = best
        self.pars = opt_params
        self.final_state = self.checkerboard(self.init_state, self.p, cycle(opt_params))

###################### Other functions ############

    def solve(self, target, depth):
        target=target/np.linalg.norm(target)

        initial = np.zeros(2**self.n) 
        initial[0]=1

        H = -np.outer(target,target.conj())

        self.ex_minimize (initial, H, depth)
        
###################### Other functions ############

    def rand_params(self, p): #returns parameters, number of parameters, and number of blocks in tha ensatz
        n = self.n
        if n % 2 == 0:
            num_params = 5*n*p
            num_blocks = n*p
        else:
            num_params = 5*(n-1)*p
            num_blocks = (n-1)*p
        params = np.random.rand(num_params)*2*pi
        return params, num_params, num_blocks

In [116]:
n_qubits = 8
var_solver = Variational_amplitude_embeding(n_qubits)

In [117]:
target = np.random.rand(2**n_qubits)+1j*np.random.rand(2**n_qubits)

In [118]:
var_solver.solve(target, depth=4)

In [119]:
-var_solver.ex_val

0.8310861092714816

In [120]:
var_solver.time_tot

365.5881609916687

In [121]:
var_solver.final_state

array([-0.05576677-9.47990518e-03j, -0.06458981+8.17639977e-03j,
       -0.04460278-2.13416861e-02j, -0.05989993+3.28875114e-03j,
       -0.06891448+2.84455018e-03j, -0.06823396+5.77297559e-03j,
       -0.06246418-7.12009566e-03j, -0.05595985+6.69182486e-03j,
       -0.06259235-1.87125689e-02j, -0.07828918+5.20732314e-03j,
       -0.06414687-2.26478713e-02j, -0.05834735-1.79091577e-02j,
       -0.05068278-3.58540680e-03j, -0.04393568-3.00782189e-03j,
       -0.04061159-1.38988677e-03j, -0.04817507+1.52819346e-02j,
       -0.05926299-9.70294948e-03j, -0.06443647+4.36014636e-04j,
       -0.05371926-1.72695466e-02j, -0.06511099+1.04903188e-02j,
       -0.06319621-1.20043077e-02j, -0.07196214-1.23260309e-02j,
       -0.05935033-2.27161474e-02j, -0.036986  +5.01223968e-05j,
       -0.0625478 -2.04472300e-02j, -0.07052175-2.42113985e-02j,
       -0.04566067-1.91545179e-02j, -0.05015556-1.54729241e-03j,
       -0.05950699-2.44417558e-02j, -0.06413764-8.13365928e-03j,
       -0.0508573 -1.0895